In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from PIL import Image
import os

In [6]:
def page_from_to(a,b):
    articleList = []
    for page in range(b, a-1, -1):
        url = 'https://www.ptt.cc/bbs/Beauty/index{}.html'.format(page)
        resp = requests.get(url, cookies={'over18': '1'})
        soup = BeautifulSoup(resp.text, 'html.parser')
        for entry in soup.find_all(class_ = 'r-ent'):
            if entry.find(class_="hl f1"):
                try:
                    hot = entry.find(class_="hl f1").text.strip()
                    title = entry.find(class_ = 'title').text.strip()
                    article_url = 'https://www.ptt.cc'+ entry.find('a')['href']
                    articleList.append({
                        'hot' : hot,
                        'title' : title,
                        'url' : article_url
                })
                except TypeError:
                    print('有一篇文章不存在')
            elif entry.find(class_="hl f3"):
                try:
                    hot = entry.find(class_="hl f3").text.strip()
                    if int(hot) >= 50:
                        title = entry.find(class_ = 'title').text.strip()
                        article_url = 'https://www.ptt.cc'+ entry.find('a')['href']
                        articleList.append({
                            'hot' : hot,
                            'title' : title,
                            'url' : article_url
                        })
                except TypeError:
                    print('有一篇文章不存在')
    articleDataFrame = pd.DataFrame.from_dict(articleList)
    articleDataFrame = articleDataFrame.sort_values(by = 'hot', ascending = False)
    return articleDataFrame

In [3]:
def download(df):
    for hot_url in df['url']:
        resp = requests.get(hot_url, cookies={'over18': '1'})
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        output_dir = 'downloads'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # 定位照片的位置
        img_tags = soup.find(id = 'main-content').findChildren('a', recursive = False)
        for img_tag in img_tags:
            if 'imgur' not in img_tag['href']:
                continue
            img_id = img_tag['href'].split('/')[-1]
            img_url = 'http://i.imgur.com/{}.jpg'.format(img_id)
    
        # 對圖片送出請求
            with requests.get(img_url, stream = True) as r:
                r.raise_for_status()
                # 檢查圖片副檔名
                img = Image.open(r.raw)
                img_savename = '{outdir}/{img_id}.{img_ext}'.format(
                    outdir=output_dir, img_id=img_id, img_ext=img.format.lower())
                img.save(img_savename)
                print('Save image {}'.format(img_savename))
